# Atomap tutorial: finding and analysing sublattices

For more details see the open access article: **Atomap: a new software tool for the automated analysis of atomic resolution images using two-dimensional Gaussian fitting**. https://dx.doi.org/10.1186/s40679-017-0042-5

This tutorial shows how to use Atomap to analyse the atomic structure of a perovskite oxide structure, by manually finding the two sublattices in an image.

More documentation is found at http://atomap.org/

*Authorship history*:

- Apr 4, 2017 - Original Notebook written by [Magnus Nord](https://www.ntnu.edu/employees/magnus.nord)
- Oct 9, 2017 - Updates by Ida Hjorth
- Feb 24, 2020 - Further updates by Magnus Nord
- Jul 23, 2022 - Update documentation by Joshua Taillon

## Importing the libraries

Firstly, we must set the plotting toolkit:

In [ ]:
%matplotlib notebook

Atomap relies heavily on HyperSpy for visualization and fitting, and uses HyperSpy signals for most of the outputs. So we need to import both HyperSpy and Atomap.

You might get a "WARNING:hyperspy_gui_traitsui", this can be ignored.

In [ ]:
# import hyperspy and atomap


### Loading data

Atomap uses HyperSpy signals as its input, which can be any loaded from many different types of files. DM3/DM4, tif, emi/ser, jpg or HDF5-files. This can be loaded using `s = hs.load(your_filename)`.

Here we will be using a test dataset, generated by the `dummy_data` module.

In [ ]:
# get the "fantasite" dummy data set


This is an imaginary structure, similar to a perovskite oxide projected along on the [110] direction.

In [ ]:
# plot the fantasite image


## Finding initial positions

Our first task is finding and fitting the most intense atomic columns. This is done by using a peak finder, where the minimum distance between the features is used as an input parameter. This function returns a HyperSpy signal, where the atom positions are saved as permanent markers in the metadata

In [ ]:
# get the separation between the most intense columns


In this plot, the minimum feature separation is shown on the x-axis for the navigation plot. This parameter can be changed by using the arrow keys. For this image, we want to get the peaks for only the most intense atomic columns.
A feature separation of 12 works fine.

In [ ]:
# plot the result of the peak finding (may be a bit slow to render)


We use this value as an input for the next step, which involves getting these atomic positions as a list.

In [ ]:
# get the atomic positions as a list


To add or remove atoms, use `add_atoms_with_gui`. Any changes will automatically be updated in the output list `atom_positions_manual`.

In [ ]:
# demonstrate manually adding/removing atoms from list


## Making a sublattice

The atom positions are used to initialize the sublattice object.

In [ ]:
# create a "SubLattice" object


This sublattice object contains the atom positions we just found, the image, and many utility functions. For example a plot function.

In [ ]:
# plot the sublattice


The first step is to refine the atomic positions using center of mass, since the initial atom positions are not very good.

In [ ]:
# refine atomic positions by finding nearest neighbors and using their center of mass



Then using 2D Gaussians

In [ ]:
# Finally refine using a 2D Gaussian model fit


This will give much more accurate atomic positions:

In [ ]:
# plot the refined sublattice


## Constructing zone axes

The next step is to find the relation between the atom positions, by ordering them into atomic planes:

In [ ]:
# order atomic positions into planes (zone axes)


These atomic planes can be visualized. Navigate using the arrow keys. The number to the top left is the direction of the zone axis.

In [ ]:
# plot the sublattice planes


## Finding the second sublattice

In this structure, we have two separate sublattices. Having found the first one, we can use those atomic positions to find the second one. Here we use the fact that the atoms in the second sublattice is located between the atoms in the second zone axis for the first sublattice.

In [ ]:
# get the average distance between atoms


Then we pass this to the `find_missing_atoms_from_zone_vector`, which returns a new list of positions

In [ ]:
# pass theses positions to find the remaining atoms that make up the second sublattice


Secondly, we "subtract" the intensity of the `sublattice` atoms from the fantasite image

In [ ]:
# subtract the intensity of the first sublattice from the image to make visualiazation/peak finding easier


Which we use to make the second sublattice:

In [ ]:
# create second sublattice and plot it



And refine the positions

In [ ]:
# refine the atomic positions for the second sublattice like done for the first




## The atom lattice object

Having made these two sublattices, we can combine them in an `Atom_Lattice` object

In [ ]:
# combine the two sublattices into a single Atom_Lattice and plot the result



Atom_Lattice objects can be stored, to avoid having to run the often slow fitting routines several times.

In [ ]:
# example of saving the atom lattice to disk
atom_lattice.save(overwrite=True)

Which can be restored:

In [ ]:
# example of loading an atom lattice from disk
atom_lattice2 = am.load_atom_lattice_from_hdf5("fantasite_atom_lattice.hdf5")

## Analysing the structure

The sublattice object contains many utility functions for visualizing various structural effects.

For example, the distance between the atoms.

In [ ]:

# get the distance between monolayers in the sublattice (may need to re-construct zone axes first)

# plot the result


Or the variations in distance between the atoms:

In [ ]:
# calculate and plot the variation in distance between atoms



Or the ellipticity of the atoms, which can be used to extract information about the structure parallel to the electron beam. Plotting only the magnitude:

In [ ]:
# calculate and plot the ellipticity of the signal



Plotting both the magnitude, and the direction:

In [ ]:
# plot the ellipticity as vectors


## Integrating intensity

The intensity of the atomic columns can be found using `integrate_column_intensity` function, which supports two different methods: `Watershed` and `Voronoi`, where the latter is the default one.

In [ ]:
# integrate the atomic column intensity parallel to the beam and plot the result



This also works for sublattices directly. Since the fantasite dataset contain several sublattices, the results here will contain intensity from several columns. So for situation like this, the `atom_lattice.integrate_column_intensity` should be used.

In [ ]:
# do the same thing but just for the first sublattice



## Line profiles

All these structural properties can also be plotted as line profiles, as a function of distance from some atomic plane. For example the distance between the atoms in the horizontal direction.

This requires two things: 1) the zone axis perpendicular to the horizontal direction, and 2) some atomic plane perpendicular to the horizontal direction.

The former is found using `plot_planes`

In [ ]:
# plot the atomic planes of the second sublattice


Which shows that the second zone axis is the one perpendicular to the horisontal axis. Which we get from `zones_axis_average_distances`. Note that Python is zero-indexed, meaning we need to get index `1`

In [ ]:
# extract the average distances in the perpendicular zone axis


Then we extract an atomic plane which is also perpendicular to the horizontal axis

In [ ]:
# extract a single atomic plan perpendicular to the horizontal axis


Lastly, we get the line profile:

In [ ]:
# Get line insensity profile and plot



This can be done for other properties as well:

In [ ]:
# do the same thing for ellipticity




## Properties

The atomic positions can be accessed directly as well

In [ ]:
# display the first sublattice's 'x_position' attribute 


These properites include: `y_position`, `sigma_x`, `sigma_y`, `ellipticity`, `rotation_ellipticity`.

These can be visualized using a variety of methods in the `sublattice` object, or stored to (for example) a csv-file which can be opened in spreadsheet software

In [ ]:
# example saving sublattice attributes  







The easiest way to save the monolayer map from earlier is through the HyperSpy HDF5 format

In [ ]:
# example saving the monolayer map as a HypseSpy signal


These signals can also be saved as tif, so they can be opened in other image viewers. For this to work, we must first change NaN values to zero:

In [ ]:
# convert NaN values to zero using NumPy



This file can be opened in image viewers such as ImageJ or Digital Micrograph

In [ ]:
# save monolayer map as a tif
s_monolayer.save("monolayer_distance_map0.tif", overwrite=True)

## Determining polarization

In many ferroelectric materials, the spontaneous electric polarization can be determined by looking at the shift of some atomic columns in relation to the others. One example of this is in the ferroelectric perovskite oxides, where the B-cation is shifted from its cubic centrosymmetric position. The polarization can be determined by finding both the direction and magnitude of this shift.

Firstly, we get an appropriate artificial dataset, resembling a ferroelectric thin film grown on top of a non-ferroelectric substrate.

In [ ]:
# open the polarization dummy data


# plot the signal and notice the atom lattice is already defined 


The blue, B-cation, atom columns in the top part of the image are shifted towards the left. By finding the centre position of four neighboring red A-cation forming a square, this shift can be quantified.

Finding these neighbors relies on moving along two zone axis directions in the A-cation sublattice.

In [ ]:
# construct zone axes in one of the cation sublattices



Next, find the two perpendicular zone axes spanning this square. For the perovskite oxide 100 projection, this is most likely the two first ones.

In [ ]:
# use the plot_planes method to find perpendicular zone axes that represent the unit cell


The zone axes are then used with the B-cation sublattice:

In [ ]:
# calculate average distances on the sublattices




# compute the polarization from the second sublattice


This can be visualized directly by using the ``plot`` method, and the data itself can be accessed in the signal’s metadata.

In [ ]:
# visualize the polarization and save the polarization values to a new variable

